# Use network isolation with managed online endpoints

Learn how to secure communication with online endpoints by using private endpoints. This script follows the [Use network isolation with managed online endpoints](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-secure-online-endpoint) document. 

The following prerequisites are needed for this example: 

Install them using the following code:

In [ ]:
%pip install azure-mgmt-resource
%pip install azure-mgmt-compute
%pip install azure-mgmt-containerregistry
%pip install azure-mgmt-network

## 1. Connect to Azure Machine Learning

### 1.1 Import Required Libraries

In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.mgmt.resource import ResourceManagementClient, DeploymentScriptsClient
from azure.mgmt.resource.templatespecs import TemplateSpecsClient
from azure.mgmt.resource.templatespecs.models import TemplateSpec
from azure.mgmt.resource.resources.v2021_04_01 import models as resource_models
from azure.mgmt.resource.templatespecs.v2022_02_01.models import TemplateSpec
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.compute.v2022_08_01.models import (
    VirtualMachineRunCommand,
    VirtualMachine,
    VirtualMachineRunCommandScriptSource,
    RunCommandInputParameter,
)
from azure.identity import DefaultAzureCredential, ManagedIdentityCredential
import os, json, random
from pathlib import Path

### 1.2 Set variables

In [4]:
# subscription = "<SUBSCRIPTION_ID>"
# resource_group = "<RESOURCE_GROUP>"
# location = "<LOCATION>"

subscription = "6fe1c377-b645-4e8e-b588-52e57cc856b2"
resource_group = "role-creation-sample"
location = "eastus"

_rand = random.randint(0, 10000)
_rand = 4038

suffix = f"vnet{_rand}"
acr_name = f"cr{suffix}"
workspace = f"mlw-{suffix}"
identity_name = f"uai{suffix}"

vm_name = f"moevnet-vm{suffix}"
vnet_name = f"vnet-{suffix}"
subnet_name = "snet-scoring"

tempspec_name = f"ts-networkiso-{_rand}"

endpoint_name = f"endpt-vnet-{_rand}"
##TODO: Edit
git_branch = os.getenv("GITHUB_HEAD_REF", "alwallace/networkisolation3")

### 1.3 Get clients

To create the secured resources needed for a network-isolated Managed Online Endpoint including the VM Jump Box, we will need a Resource Management Client and a Template Specs client. Once the resources are created, we will use a ComputeManagementClient to connect to the VM Jump Box and issue commands to the workspace.

In [5]:
credential = DefaultAzureCredential()

resource_client = ResourceManagementClient(credential, subscription)

compute_client = ComputeManagementClient(credential, subscription)

ts_client = TemplateSpecsClient(credential, subscription)

## 2. Create the workspace and secured resources

To create the workspace and associated resources, we will use the ARM templates located in [vnet/setup_vm](vnet/setup_vm) and [vnet/setup_ws](vnet/setup_ws). These ARM templates were created by decompiling the Bicep files used in the Network Isolation CLI example directory [cli/endpoints/online/managed/vnet](../../../../../cli/endpoints/online/managed/vnet/setup_ws).

The Bicep decompilation tool is distributed as binary executable and can be installed as an Azure CLI extension or a standalone tool. For more information, see the document [Install Bicep Tools](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/install).

### 2.1 Create identity

To create the UAI identity we use the Resource Client to deploy the [uai.json](vnet/setup_ws/uai.json) ARM template by passing it as the `template` property with parameters passed in under the `parameters` property.

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=f"{suffix}-uai",
    parameters={
        "properties": {
            "mode": "Incremental",
            "template": json.loads(Path("vnet/setup_ws/uai.json").read_text()),
            "parameters": {
                "suffix": {"value": f"{suffix}"},
            },
            "location": {"value": f"{location}"},
        }
    },
).result()

### 2.2 Create workspace

The ARM template to create the workspace consists of a main tmplate in [vnet/setup_ws/main.json](vnet/setup_ws/main-linked.json) which refers to other templates in the [modules](vnet/setup_ws/modules) directory. To deploy these linked templates, a Template Spec is first created which contains each of the linked modules

### 2.2.1 Create a Template Spec
A Template Spec with the name "networkiso" is created.

In [ ]:
tempspec_root = Path("vnet/setup_ws")

tempspec = ts_client.template_specs.create_or_update(
    template_spec_name=tempspec_name,
    resource_group_name=resource_group,
    template_spec={"location": location},
)

### 2.2.2 Create a Template Spec Version
A Template Spec Version is created which contains the main and linked templates. Each linked template contains an arbitrary `path` value which is independent of its actual location on disk, and should correspond to the `relativePath` values specified in the main template. The `main-linked.json` file expects relative paths of the form `modules/*.json`, so this path is provided for each linked template. 

Once the Template Spec Version is created, you can view it in the Portal.

In [ ]:
tempspec = ts_client.template_spec_versions.create_or_update(
    template_spec_name=tempspec_name,
    template_spec_version=str(_rand),
    resource_group_name=resource_group,
    template_spec_version_model={
        "location": location,
        "linked_templates": [
            {
                "template": json.loads(f.read_text()),
                "path": str(f.relative_to(tempspec_root)),
            }
            for f in (tempspec_root / "modules").iterdir()
        ],
        "main_template": json.loads((tempspec_root / "main-linked.json").read_text()),
    },
)

### 2.2.3 Deploy the Template Spec Version

Deploying the Template Spec Version creates the resources specified in the template.

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=suffix,
    parameters={
        "properties": {
            "mode": "Incremental",
            "template_link": {"id": tempspec.id},
            "parameters": {"suffix": {"value": suffix}},
        }
    },
).result()

## 3. Create the Virtual Machine Jump Box

A Virtual Machine Jump Box is to interact with the Network Isolated Machine Learning Workspace. 

To deploy the Virtual Machine Jump box, we will use a non-linked ARM template which can be passed directly to the Resource Client without first creating a Template Spec.

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=f"{vm_name}-{endpoint_name}",
    parameters={
        "properties": {
            "mode": "Incremental",
            "template": json.loads(Path("vnet/setup_vm/vm-main.json").read_text()),
            "parameters": {
                "vmName": {"value": vm_name},
                "vnetName": {"value": vnet_name},
                "identityName": {"value": identity_name},
                "subnetName": {"value": subnet_name},
            },
        }
    },
).result()

## 4. Configure the VM
The following scripts use the `RunShellScript` VM command to run shell and Python scripts that setup the VM, create a Managed Online Endpoint, and score it. 

Alternatively, these scripts can also be run over SSH.

### 4.1 Execute the setup script

The following `RunCommand` executes the setup script [vmsetup.sh](vnet/setup_vm/modules/scripts/vmsetup.sh) on the VM Jump Box. This script bootstraps the environment on the VM by setting up Python and required packages and cloning the environemnt.

In [6]:
command = compute_client.virtual_machines.begin_run_command(
    resource_group_name=resource_group,
    vm_name=vm_name,
    parameters={
        "script": [Path("vnet/setup_vm/scripts/vmsetup.sh").read_text()],
        "command_id": "RunShellScript",
        "parameters": [
            {"name": "GIT_BRANCH", "value": git_branch},
        ],
    },
).result()

ResourceExistsError: (OperationNotAllowed) The operation requires the VM to be running (or set to run).
Code: OperationNotAllowed
Message: The operation requires the VM to be running (or set to run).

In [ ]:
command.as_dict()

### 4.2 Build Image

After the initial VM setup, each Python script is executed using the [run_py.sh](vnet/setup_vm/run_py.sh) shell script.

The following `RunCommand` executes the [build_image.py](vnet/setup_vm/scripts/build_image.py) Python script on the VM Jump Box.

In [ ]:
command = compute_client.virtual_machines.begin_run_command(
    resource_group_name=resource_group,
    vm_name=vm_name,
    parameters={
       "script": [Path("vnet/setup_vm/scripts/run_py.sh").read_text()],
        "command_id": "RunShellScript",
        "parameters": [
            {"name": "SCRIPT_NAME", "value": "build_image.py"},
            {"name": "SUBSCRIPTION_ID", "value": subscription},
            {"name": "RESOURCE_GROUP", "value": resource_group},
            {"name": "WORKSPACE", "value": workspace},
            {"name": "CONTAINER_REGISTRY", "value": acr_name},
            {"name": "IMAGE_NAME", "value": "sample-image"},
            {"name": "ENV_DIR_PATH", "value": "vnet/sample/environment"},
            {"name": "LOCATION", "value": location},
            {"name": "VNET_NAME", "value": vnet_name},
        ],
    },
).result()

In [31]:
vnet_name

'vnet-vnet836'

## 5. Create a secured Managed Online Endpoint

### 5.1 Create the endpoint

The following `RunCommand` executes the [build_image.py](vnet/setup_vm/scripts/build_image.py) Python script on the VM Jump Box.

In [ ]:
command = compute_client.virtual_machines.begin_run_command(
    resource_group_name=resource_group,
    vm_name=vm_name,
    parameters={
        "script": [Path("vnet/setup_vm/scripts/run_py.sh").read_text()],
        "command_id": "RunShellScript",
        "parameters": [
            {"name": "SCRIPT_NAME", "value": "create_moe.py"},
            {"name": "SUBSCRIPTION", "value": subscription},
            {"name": "RESOURCE_GROUP", "value": resource_group},
            {"name": "WORKSPACE", "value": workspace},
            {"name": "IMAGE_NAME", "value": "sample-image"},
            {"name": "CONTAINER_REGISTRY", "value": acr_name},
            {"name": "ENDPOINT_NAME", "value": endpoint_name},
        ],
    },
).result()

In [ ]:
endpoint_name

In [ ]:
Path("vnet/setup_vm/scripts/run_py.sh").read_text()

In [ ]:
command.as_dict()

### 5.2 Make a scoring request

The following `RunCommand` executes the [score_moe.py](vnet/setup_vm/scripts/score_moe.py) Python script on the VM Jump Box.

In [ ]:
command = compute_client.virtual_machines.begin_run_command(
    resource_group_name=resource_group,
    vm_name=vm_name,
    parameters={
        "source": {"script": Path("vnet/setup_vm/scripts/run_py.sh").read_text()},
        "command_id": "RunShellScript",
        "parameters": [
            {"name": "SCRIPT_NAME", "value": "create_moe.py"},
            {"name": "SUBSCRIPTION", "value": subscription},
            {"name": "RESOURCE_GROUP", "value": resource_group},
            {"name": "WORKSPACE", "value": workspace},
            {"name": "LOCATION", "value": location},
            {"name": "CONTAINER_REGISTRY", "value": acr_name},
            {"name": "IDENTITY_NAME", "value": identity_name},
            {"name": "SAMPLE_REQUEST_PATH", "value": "vnet/sample/sample-request.json"},
        ],
    },
).result()

In [ ]:
workspace

## 6. Cleanup

### 6.1 Delete the endpoint

The following `RunCommand` executes the [delete_moe.py](vnet/setup_vm/scripts/delete_moe.py) Python script on the VM Jump Box.

In [ ]:
command = compute_client.virtual_machines.begin_run_command(
    resource_group_name=resource_group,
    vm_name=vm_name,
    parameters={
        "source": {"script": Path("vnet/setup_vm/scripts/run_py.sh").read_text()},
        "command_type": "RunShellScript",
        "parameters": [
            {"name": "SCRIPT_NAME", "value": "delete_moe.py"},
            {"name": "SUBSCRIPTION", "value": subscription},
            {"name": "RESOURCE_GROUP", "value": resource_group},
            {"name": "WORKSPACE", "value": workspace},
            {"name": "ENDPOINT_NAME", "value": endpoint_name},
        ],
    },
).result()

### 6.2 Delete the VM

In [ ]:
compute_client.virtual_machines.begin_delete(
    resource_group_name=resource_group, vm_name=vm_name
)